In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV


import tensorflow as tf
from tensorflow import keras
from hyperopt import fmin, tpe, hp
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


from sklearn.ensemble import AdaBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_classification
from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline



In [84]:
df = pd.read_csv('dataset.csv')
df.rename(columns = {'Reached.on.Time_Y.N' : 'Reached'}, inplace = True)
df = df.drop('ID', axis = 1)
df.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached
0,D,Flight,4,2,177,3,low,F,44,1233,1
1,F,Flight,4,5,216,2,low,M,59,3088,1
2,A,Flight,2,2,183,4,low,M,48,3374,1
3,B,Flight,3,3,176,4,medium,M,10,1177,1
4,C,Flight,2,2,184,3,medium,F,46,2484,1


In [85]:
df1 = pd.get_dummies(df, columns=['Warehouse_block', 'Mode_of_Shipment'], dtype = int)
df1.drop(['Mode_of_Shipment_Ship', 'Warehouse_block_F'], axis = 1, inplace = True)
df1['Product_importance'] = df1['Product_importance'].apply(lambda x : 3 if x == 'high' else (2 if x == 'medium' else 1))
df1['Gender'] = df1['Gender'].apply(lambda x : 1 if x == 'M' else 0)
df1


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Mode_of_Shipment_Flight,Mode_of_Shipment_Road
0,4,2,177,3,1,0,44,1233,1,0,0,0,1,1,0
1,4,5,216,2,1,1,59,3088,1,0,0,0,0,1,0
2,2,2,183,4,1,1,48,3374,1,1,0,0,0,1,0
3,3,3,176,4,2,1,10,1177,1,0,1,0,0,1,0
4,2,2,184,3,2,0,46,2484,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,4,1,252,5,2,0,1,1538,1,1,0,0,0,0,0
10995,4,1,232,5,2,0,6,1247,0,0,1,0,0,0,0
10996,5,4,242,5,1,0,4,1155,0,0,0,1,0,0,0
10997,5,2,223,6,2,1,2,1210,0,0,0,0,0,0,0


In [81]:
# df1.to_csv('Encoded.csv', index = False)

In [86]:
X = df1.drop('Reached', axis = 1)
y = df1['Reached']

In [56]:
class ModelTrainer:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.scaler = StandardScaler()
        self.X_train_scaled = self.scaler.fit_transform(self.X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)

    def train_logistic_regression(self):
        param_grid = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}
        lr = LogisticRegression(solver='liblinear')
        grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='f1')
        grid_search.fit(self.X_train_scaled, self.y_train)
        self.lr_model = grid_search.best_estimator_
        self.lr_f1 = f1_score(self.y_test, self.lr_model.predict(self.X_test_scaled))

    def train_svm(self):
        param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly']}
        svm = SVC()
        grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='f1')
        grid_search.fit(self.X_train_scaled, self.y_train)
        self.svm_model = grid_search.best_estimator_
        self.svm_f1 = f1_score(self.y_test, self.svm_model.predict(self.X_test_scaled))

    def train_naive_bayes(self):
        nb = GaussianNB()
        nb.fit(self.X_train_scaled, self.y_train)
        self.nb_model = nb
        self.nb_f1 = f1_score(self.y_test, self.nb_model.predict(self.X_test_scaled))

    def train_decision_tree(self):
        param_grid = {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
        dt = DecisionTreeClassifier()
        grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='f1')
        grid_search.fit(self.X_train_scaled, self.y_train)
        self.dt_model = grid_search.best_estimator_
        self.dt_f1 = f1_score(self.y_test, self.dt_model.predict(self.X_test_scaled))

    def train_random_forest(self):
        param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
        rf = RandomForestClassifier()
        grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='f1')
        grid_search.fit(self.X_train_scaled, self.y_train)
        self.rf_model = grid_search.best_estimator_
        self.rf_f1 = f1_score(self.y_test, self.rf_model.predict(self.X_test_scaled))

    def train_xgboost(self):
        param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.1, 0.2]}
        xgb = XGBClassifier()
        grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='f1')
        grid_search.fit(self.X_train_scaled, self.y_train)
        self.xgb_model = grid_search.best_estimator_
        self.xgb_f1 = f1_score(self.y_test, self.xgb_model.predict(self.X_test_scaled))

# Example usage:
if __name__ == "__main__":

    
    trainer = ModelTrainer(X, y)

    print('#'*20)
    trainer.train_logistic_regression()
    print("Logistic Regression F1 Score:", trainer.lr_f1)

    print('#'*20)
    trainer.train_svm()
    print("SVM F1 Score:", trainer.svm_f1)

    print('#'*20)
    trainer.train_naive_bayes()
    print("Naive Bayes F1 Score:", trainer.nb_f1)

    print('#'*20)
    trainer.train_decision_tree()
    print("Decision Tree F1 Score:", trainer.dt_f1)

    print('#'*20)
    trainer.train_random_forest()
    print("Random Forest F1 Score:", trainer.rf_f1)
    
    print('#'*20)
    trainer.train_xgboost()
    print("XGBoost F1 Score:", trainer.xgb_f1)
    

####################
Logistic Regression F1 Score: 0.6988876102800153
####################


KeyboardInterrupt: 

In [ ]:
class ModelTrainer_1:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.scaler = StandardScaler()
        self.X_train_scaled = self.scaler.fit_transform(self.X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)

    def train_random_forest(self):
        param_dist = {
            'n_estimators': np.arange(100, 1000, 100),
            'max_features': ['auto', 'sqrt'],
            'max_depth': [None] + list(np.arange(10, 110, 10)),
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'bootstrap': [True, False]
        }
        rf = RandomForestClassifier(random_state=42)
        random_search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=100, cv=5, scoring='f1', n_jobs=-1, random_state=42)
        random_search.fit(self.X_train_scaled, self.y_train)
        self.rf_model = random_search.best_estimator_
        self.rf_f1 = f1_score(self.y_test, self.rf_model.predict(self.X_test_scaled))

# Example usage:
if __name__ == "__main__":

    trainer = ModelTrainer_1(X, y)
    trainer.train_random_forest()

    print("Random Forest F1 Score:", trainer.rf_f1)
    print("Best Random Forest Parameters:", trainer.rf_model.get_params())

/Users/rachitkumarsingh/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
230 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rachitkumarsingh/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rachitkumarsingh/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/rachitkumarsingh/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 637, in _v

Random Forest F1 Score: 0.6944894651539709
Best Random Forest Parameters: {'bootstrap': False, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 80, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 700, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [ ]:
class ModelTrainer:
    def __init__(self, model, param_grid):
        self.model = model
        self.param_grid = param_grid
        self.best_model = None

    def train(self, X_train, y_train):
        grid_search = RandomizedSearchCV(self.model, self.param_grid, scoring='f1', cv=5, n_iter=10, random_state=42, error_score='raise')
        grid_search.fit(X_train, y_train)
        self.best_model = grid_search.best_estimator_
        return self.best_model

    def predict(self, X_test):
        return self.best_model.predict(X_test)

def main(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define models and hyperparameter grids
    models = {
        'Logistic Regression': (LogisticRegression(),
                                {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2']}),
        
        'SVM': (SVC(),
                {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'],
                 'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1]}),

        'Naive Bayes': (GaussianNB(), {}),

        'Decision Tree': (DecisionTreeClassifier(),
                          {'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30, 40, 50],
                           'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}),

        'Random Forest': (RandomForestClassifier(),
                          {'n_estimators': [50, 100, 200, 400], 'max_depth': [None, 10, 20, 30, 40, 50],
                           'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}),

        'XGBoost': (xgb.XGBClassifier(),
                    {'n_estimators': [50, 100, 200, 400], 'max_depth': [3, 4, 5, 6],
                     'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.8, 0.9, 1.0]})
    }

    best_models = {}
    
    for model_name, (model, param_grid) in models.items():
        trainer = ModelTrainer(model, param_grid)
        best_model = trainer.train(X_train_scaled, y_train)
        y_pred = trainer.predict(X_test_scaled)
        f1 = f1_score(y_test, y_pred)
        
        print(f'{model_name} - Best F1 Score: {f1:.4f} - Best Hyperparameters: {trainer.best_model.get_params()}')
        
        best_models[model_name] = trainer.best_model
    
    best_model_name = max(best_models, key=lambda k: f1_score(y_test, best_models[k].predict(X_test_scaled)))
    best_model = best_models[best_model_name]
    
    print(f'Best Model: {best_model_name} - Best F1 Score: {f1:.4f}')

if __name__ == "__main__":
    main(X,y)

/Users/rachitkumarsingh/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Logistic Regression - Best F1 Score: 0.7246 - Best Hyperparameters: {'C': 0.001, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
SVM - Best F1 Score: 0.7447 - Best Hyperparameters: {'C': 0.001, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Naive Bayes - Best F1 Score: 0.5993 - Best Hyperparameters: {'priors': None, 'var_smoothing': 1e-09}


/Users/rachitkumarsingh/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Decision Tree - Best F1 Score: 0.6959 - Best Hyperparameters: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}
Random Forest - Best F1 Score: 0.6961 - Best Hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
XGBoost - Best F1 Score: 0.7130 - Best Hyperparameters: {'objective': 'binary:logistic', 'use_label_encoder': None, 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsampl

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [75]:
class DeepLearningModel(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, hidden_units=128, activation='relu'):
        self.input_dim = input_dim
        self.hidden_units = hidden_units
        self.activation = activation
        self.model = Sequential()
        self.build_model()

    def build_model(self):
        self.model.add(Dense(self.hidden_units, input_dim=self.input_dim, activation=self.activation))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    def fit(self, X, y, epochs=10, batch_size=32):
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size)

    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype(int)

# Define an AdaBoostClassifier with the deep learning model
class AdaBoostDLClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator=None, n_estimators=50, learning_rate=1.0):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.model = None

    def fit(self, X, y):
        base_estimator = self.base_estimator or DeepLearningModel(input_dim=X.shape[1])
        self.model = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=self.n_estimators, learning_rate=self.learning_rate)
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

# Hyperparameter optimization using GridSearchCV
def optimize_hyperparameters(X, y):
    param_grid = {
        'base_estimator__base_estimator__hidden_units': [64, 128],
        'base_estimator__base_estimator__activation': ['relu', 'tanh'],
        'n_estimators': [50, 100],
        'learning_rate': [0.1, 0.5]
    }

    ada_boost_model = AdaBoostDLClassifier()

    grid_search = GridSearchCV(ada_boost_model, param_grid, cv=3, scoring='f1')
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    return best_params, best_score

# Train and evaluate the model
best_params, best_score = optimize_hyperparameters(X_train, y_train)
print("Best Hyperparameters:", best_params)
print("Best F1 Score:", best_score)

# Train the final model with the best hyperparameters
final_model = AdaBoostDLClassifier(base_estimator=DeepLearningModel(input_dim=X_train.shape[1], **best_params))
final_model.fit(X_train, y_train)

# Evaluate the final model
y_pred = final_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("Final F1 Score on Test Data:", f1)

AttributeError: 'NoneType' object has no attribute 'set_params'

In [79]:
space = {
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'dropout_rate': hp.uniform('dropout_rate', 0.2, 0.5),
    'hidden_units': hp.choice('hidden_units', [64, 128, 256]),
    'epochs': hp.choice('epochs', [10, 20, 30]),
    'batch_size': hp.choice('batch_size', [32, 64, 128])
}

# Define the deep learning model
def create_model(params):
    model = keras.Sequential([
        keras.layers.Input(shape=(X_train.shape[1],)),
        keras.layers.Dense(params['hidden_units'], activation='relu'),
        keras.layers.Dropout(params['dropout_rate']),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    
    optimizer = keras.optimizers.Adam(learning_rate=params['learning_rate'])
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the objective function to minimize (cross-validation)
def objective(params):
    model = create_model(params)
    history = model.fit(X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'], verbose=0)
    _, val_accuracy = model.evaluate(X_test, y_test, verbose=0)
    return -val_accuracy  # Negative accuracy to maximize

# Hyperparameter optimization using Hyperopt
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, verbose=1, rstate=np.random.seed(42))

# Print the best hyperparameters found
print("Best Hyperparameters:")
print(best)

# Train the final model with the best hyperparameters
best_model = create_model(best)
best_model.fit(X_train, y_train, epochs=best['epochs'], batch_size=best['batch_size'], verbose=1)

# Evaluate the final model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

  2%|▏         | 1/50 [00:01<01:27,  1.79s/trial, best loss: -0.6613636612892151]

  4%|▍         | 2/50 [00:03<01:14,  1.56s/trial, best loss: -0.6613636612892151]

  6%|▌         | 3/50 [00:04<00:59,  1.27s/trial, best loss: -0.6613636612892151]

  8%|▊         | 4/50 [00:06<01:18,  1.70s/trial, best loss: -0.6613636612892151]

...

 94%|█████████▍| 47/50 [01:14<00:05,  1.74s/trial, best loss: -0.6759091019630432]

 96%|█████████▌| 48/50 [01:15<00:03,  1.63s/trial, best loss: -0.6759091019630432]

 98%|█████████▊| 49/50 [01:17<00:01,  1.52s/trial, best loss: -0.6759091019630432]

100%|██████████| 50/50 [01:18<00:00,  1.56s/trial, best loss: -0.6759091019630432]


Best Hyperparameters:
{'batch_size': 0, 'dropout_rate': 0.43129036086834394, 'epochs': 0, 'hidden_units': 0, 'learning_rate': 0.008138714482545035}
69/69 [==============================] - 0s 381us/step - loss: 0.6931 - accuracy: 0.4068
Test Accuracy: 40.68%


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [88]:
svm_model = SVC()

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],           # Regularization parameter
    'kernel': ['linear', 'rbf'],      # Kernel type (linear or radial basis function)
    'gamma': ['scale', 'auto', 0.1, 1]  # Kernel coefficient (for 'rbf' kernel)
}

# Use RandomizedSearchCV for hyperparameter optimization
random_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=10, scoring='accuracy', cv=5, random_state=42)
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_

# Train a new SVM model with the best hyperparameters
best_svm_model = SVC(**best_params)
best_svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_svm_model.predict(X_test)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the test set:", accuracy)


Accuracy on the test set: 0.6695454545454546


# Model Pipeline

In [91]:
class ModelTrainer:
    def __init__(self, model, param_grid):
        self.model = model
        self.param_grid = param_grid
        self.best_model = None

    def train(self, X_train, y_train):
        grid_search = RandomizedSearchCV(self.model, self.param_grid, scoring='f1', cv=5, n_iter=10, random_state=42, error_score='raise')
        grid_search.fit(X_train, y_train)
        self.best_model = grid_search.best_estimator_
        return self.best_model

    def predict(self, X_test):
        return self.best_model.predict(X_test)

def main(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define models and hyperparameter grids
    models = {
        'Logistic Regression': (LogisticRegression(),
                                {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2']}),
        
        'SVM': (SVC(),
                {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'],
                 'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1]})
    }

    best_models = {}
    
    for model_name, (model, param_grid) in models.items():
        trainer = ModelTrainer(model, param_grid)
        best_model = trainer.train(X_train_scaled, y_train)
        y_pred = trainer.predict(X_test_scaled)
        f1 = f1_score(y_test, y_pred)
        
        print(f'{model_name} - Best F1 Score: {f1:.4f} - Best Hyperparameters: {trainer.best_model.get_params()}')
        
        best_models[model_name] = trainer.best_model
    
    best_model_name = max(best_models, key=lambda k: f1_score(y_test, best_models[k].predict(X_test_scaled)))
    best_model = best_models[best_model_name]
    
    print(f'Best Model: {best_model_name} - Best F1 Score: {f1:.4f}')

if __name__ == "__main__":
    main(X,y)

/Users/rachitkumarsingh/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Logistic Regression - Best F1 Score: 0.7246 - Best Hyperparameters: {'C': 0.001, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
SVM - Best F1 Score: 0.7447 - Best Hyperparameters: {'C': 0.001, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Best Model: SVM - Best F1 Score: 0.7447


In [90]:
pipe_lr=make_pipeline(df1,lr)

NameError: name 'column_trans' is not defined